In [466]:
from SmartApi.smartConnect import SmartConnect
import pyotp
import pandas as pd
import requests
import pandas as pd
from datetime import datetime, timedelta
from datetime import datetime, timedelta
import pandas_ta as ta


api_key = 'C46AImmE'
clientId = 'M507690'
pwd = '4663'
smartApi = SmartConnect(api_key)
token = "4ACNNSIZL6EV6UQ2ACSI23H5VQ"
totp=pyotp.TOTP(token).now()
correlation_id = "abc123"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

#fetch User Profile
userProfile= smartApi.getProfile(refreshToken)
User= pd.DataFrame(userProfile)
User = User['data']['name']
User
print(f"Welcome {User} to the World of SMART API BY ANGEL.....!!")
print(f"Login Sucessfull....!!")


[I 251221 01:28:10 smartConnect:124] in pool


Welcome MANA  BHATTACHARYYA to the World of SMART API BY ANGEL.....!!
Login Sucessfull....!!


In [467]:
# ---- NIFTY 50 Token (Angel One Fixed Token) ----
NIFTY_TOKEN = "26000"      # NIFTY 50 Index
EXCHANGE = "NSE"

# ---- Fetch LTP ----
ltp_data = smartApi.ltpData(
    exchange=EXCHANGE,
    tradingsymbol="NIFTY",
    symboltoken=NIFTY_TOKEN
)

print("📈 NIFTY 50 LTP:", ltp_data['data']['ltp'])

ATM = round(ltp_data['data']['ltp'] / 50) * 50
print("📈 NIFTY 50 SPOT Price:", ATM)

ltp =  ltp_data['data']['ltp']
prev_close = ltp_data['data']['close']

change = ltp - prev_close
pct_change = (change / prev_close) * 100

print(f"NIFTY Change: {change:.2f}")
print(f"NIFTY % Change: {pct_change:.2f}%")


📈 NIFTY 50 LTP: 25966.4
📈 NIFTY 50 SPOT Price: 25950
NIFTY Change: 150.85
NIFTY % Change: 0.58%


In [468]:
import pandas as pd
import pandas_ta as ta

def add_indicators(df):
    df["VWAP"] = (df["Close"] * df["Volume"]).cumsum() / df["Volume"].cumsum()
    df["EMA14"] = ta.ema(df["Close"], length=20)
    df["RSI"] = ta.rsi(df["Close"], length=14)

    macd = ta.macd(df["Close"])
    df["MACD"] = macd["MACD_12_26_9"]
    df["MACD_SIGNAL"] = macd["MACDs_12_26_9"]

    return df

In [469]:
# ---------- FETCH 5-MIN CANDLES ---------- #
to_date = datetime.now()
from_date = to_date - timedelta(days=5)

params = {
    "exchange": "NSE",
    "symboltoken": "99926000",
    "interval": "THREE_MINUTE",
    "fromdate": from_date.strftime("%Y-%m-%d %H:%M"),
    "todate": to_date.strftime("%Y-%m-%d %H:%M")
}

data = smartApi.getCandleData(params)
NIFTY = pd.DataFrame(
    data["data"],
    columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]
)
NIFTY["Datetime"] = pd.to_datetime(NIFTY["Datetime"])
NIFTY['Datetime'] = pd.to_datetime(NIFTY['Datetime']).dt.tz_localize(None)
NIFTY['NIFTY_Change'] = NIFTY['Close'].diff()

NIFTY = add_indicators(NIFTY)
NIFTY = NIFTY.tail(3)
NIFTY

,Datetime,Open,High,Low,Close,Volume,NIFTY_Change,VWAP,EMA14,RSI,MACD,MACD_SIGNAL
497,2025-12-19 15:21:00,25969.5,25974.25,25968.55,25970.20,0,1.05,NaN,25969.124722,51.682583,0.184301,0.485228
498,2025-12-19 15:24:00,25970.5,25973.65,25961.95,25965.65,0,-4.55,NaN,25968.793796,46.974384,-0.092907,0.369601
499,2025-12-19 15:27:00,25965.5,25972.20,25955.60,25961.40,0,-4.25,NaN,25968.089625,43.031123,-0.648065,0.166068


In [470]:
url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
instruments = pd.DataFrame(requests.get(url).json())
instruments.to_csv('Token_df.csv')

expiry = instruments[(instruments['name']=='NIFTY') &  (instruments['instrumenttype']=='OPTIDX') &  (instruments['exch_seg']=='NFO')]["expiry"].unique()

expiry

today = datetime.today()

dates = pd.to_datetime(expiry, format="%d%b%Y")
next_expiry = dates[dates > today].min()
print("Next Expiry:", next_expiry.strftime("%d%b%y").upper())

Next Expiry: 23DEC25


In [471]:
Symbol = 'India VIX'

def get_option_token(symbol):
    row = instruments[instruments["symbol"] == symbol]
    if row.empty:
        raise Exception(f"Token not found for {symbol}")
    return row.iloc[0]["token"]

VIX_token = get_option_token(Symbol)
print("INDIA-VIX Token:", VIX_token)


INDIA-VIX Token: 99926017


In [472]:
# ---------- FETCH 5-MIN CANDLES ---------- #
to_date = datetime.now()
from_date = to_date - timedelta(days=5)

params = {
    "exchange": "NSE",
    "symboltoken": VIX_token,
    "interval": "THREE_MINUTE",
    "fromdate": from_date.strftime("%Y-%m-%d %H:%M"),
    "todate": to_date.strftime("%Y-%m-%d %H:%M")
}

data = smartApi.getCandleData(params)
VIX = pd.DataFrame(
    data["data"],
    columns=["Datetime", "Open", "High", "Low", "Close", "Volume"]
)
VIX["Datetime"] = pd.to_datetime(VIX["Datetime"])
VIX['Datetime'] = pd.to_datetime(VIX['Datetime']).dt.tz_localize(None)
VIX['VIX_Change'] = VIX['Close'].diff()
VIX = add_indicators(VIX)
VIX.tail(2)

,Datetime,Open,High,Low,Close,Volume,VIX_Change,VWAP,EMA14,RSI,MACD,MACD_SIGNAL
498,2025-12-19 15:24:00,9.56,9.58,9.54,9.57,0,0.00,NaN,9.529069,58.973021,0.012646,0.002745
499,2025-12-19 15:27:00,9.56,9.59,9.52,9.53,0,-0.04,NaN,9.529157,50.706399,0.011001,0.004397


In [473]:
expiry_next = next_expiry.strftime("%d%b%y").upper()

def get_option_symbol(expiry_next, strike, opt_type):
    return f"NIFTY{expiry_next}{strike}{opt_type}"

ce_symbol = get_option_symbol(expiry_next, ATM, "CE")
pe_symbol = get_option_symbol(expiry_next, ATM, "PE")

print("CE Symbol:", ce_symbol)
print("PE Symbol:", pe_symbol)

CE Symbol: NIFTY23DEC2525950CE
PE Symbol: NIFTY23DEC2525950PE


In [474]:
def get_option_token(symbol):
    row = instruments[instruments["symbol"] == symbol]
    if row.empty:
        raise Exception(f"Token not found for {symbol}")
    return row.iloc[0]["token"]

ce_token = get_option_token(ce_symbol)
pe_token = get_option_token(pe_symbol)

print("CE Token:", ce_token)
print("PE Token:", pe_token)

CE Token: 57003
PE Token: 57004


NIFTY CE SIDE DATA PREDICTION

In [475]:
def fetch_5min_candles(token):
    to_date = datetime.now()
    from_date = to_date - timedelta(days=3)

    response = smartApi.getCandleData({
        "exchange": "NFO",
        "symboltoken": token,
        "interval": "THREE_MINUTE",
        "fromdate": from_date.strftime("%Y-%m-%d %H:%M"),
        "todate": to_date.strftime("%Y-%m-%d %H:%M")
    })

    if not response or "data" not in response or response["data"] is None:
        raise Exception("No candle data")

    df = pd.DataFrame(
        response["data"],
        columns=["Datetime", "open", "high", "low", "close", "volume"]
    )

    # 🔑 CRITICAL FIX
    df["Datetime"] = pd.to_datetime(df["Datetime"])
    df.set_index("Datetime", inplace=True)
    df.sort_index(inplace=True)

    return df



def apply_indicators(df):
    df = df.copy()

    df["EMA14"] = ta.ema(df["close"], length=5)
    df["EMA20"] = ta.ema(df["close"], length=10)
    df["RSI"] = ta.rsi(df["close"], length=14)

    df["VWAP"] = ta.vwap(
        high=df["high"],
        low=df["low"],
        close=df["close"],
        volume=df["volume"]
    )

    df.dropna(inplace=True)
    return df

ce_df = fetch_5min_candles(ce_token)
pe_df = fetch_5min_candles(pe_token)

ce_df = apply_indicators(ce_df)
pe_df = apply_indicators(pe_df)

In [476]:
ce_df = apply_indicators(ce_df).reset_index()
ce_df = ce_df.to_csv('CE_Data.csv')
ce_df = pd.read_csv('CE_Data.csv')
ce_df["Datetime"] = pd.to_datetime(ce_df["Datetime"])
ce_df['Datetime'] = pd.to_datetime(ce_df['Datetime']).dt.tz_localize(None)
ce_df.reset_index()
latest = ce_df.tail(3)
latest

,Unnamed: 0,Datetime,open,high,low,close,volume,EMA14,EMA20,RSI,VWAP
229,229,2025-12-19 15:21:00,91.05,91.70,90.35,91.15,19385,91.349446,91.352749,50.676140,90.829678
230,230,2025-12-19 15:24:00,91.00,91.90,89.55,91.35,27162,91.349631,91.352249,51.154245,90.830205
231,231,2025-12-19 15:27:00,91.15,96.75,91.10,95.00,40312,92.566420,92.015476,58.970673,90.856078


In [477]:
for df in [ce_df, NIFTY, VIX]:
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df['Datetime'] = df['Datetime'].dt.tz_localize(None)


df.rename(columns={'Date': 'Datetime'}, inplace=True)

nifty_sel = NIFTY[['Datetime', 'Close', 'NIFTY_Change']] \
    .rename(columns={'Close': 'NIFTY_Close'})

vix_sel = VIX[['Datetime', 'Close', 'VIX_Change']] \
    .rename(columns={'Close': 'VIX_Close'})

merged_df = ce_df.merge(
    nifty_sel,
    on='Datetime',
    how='left'
)

merged_df = merged_df.merge(
    vix_sel,
    on='Datetime',
    how='left'
)

merged_df[['NIFTY_Close','NIFTY_Change','VIX_Close','VIX_Change']] = merged_df[['NIFTY_Close','NIFTY_Change','VIX_Close','VIX_Change']].ffill()
merged_df.tail(2)

,Unnamed: 0,Datetime,open,high,low,close,volume,EMA14,EMA20,RSI,VWAP,NIFTY_Close,NIFTY_Change,VIX_Close,VIX_Change
230,230,2025-12-19 15:24:00,91.00,91.90,89.55,91.35,27162,91.349631,91.352249,51.154245,90.830205,25965.65,-4.55,9.57,0.00
231,231,2025-12-19 15:27:00,91.15,96.75,91.10,95.00,40312,92.566420,92.015476,58.970673,90.856078,25961.40,-4.25,9.53,-0.04


In [478]:
merged_df.columns

Index(['Unnamed: 0', 'Datetime', 'open', 'high', 'low', 'close', 'volume',
       'EMA14', 'EMA20', 'RSI', 'VWAP', 'NIFTY_Close', 'NIFTY_Change',
       'VIX_Close', 'VIX_Change'],
      dtype='object')

In [479]:
# Ensure sorted
df = merged_df
df = df.sort_values('Datetime').reset_index(drop=True)

# Next candle NIFTY change
df['NIFTY_Change_Future'] = df['NIFTY_Close'].shift(-1) - df['NIFTY_Close']

# Direction label
df['target'] = 0
df.loc[df['NIFTY_Change_Future'] > 5, 'target'] = 1    # BUY CE
df.loc[df['NIFTY_Change_Future'] < -5, 'target'] = -1  # BUY PE


In [480]:
features = [
    'EMA14', 'EMA20', 'RSI', 'VWAP',
    'close', 'volume',
    'NIFTY_Change',
    'VIX_Close', 'VIX_Change'
]


In [481]:
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit

X = df[features].fillna(0)
y = df['target']

tscv = TimeSeriesSplit(n_splits=5)

model = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softmax',
    num_class=3,
    eval_metric='mlogloss'
)

for train_idx, test_idx in tscv.split(X):
    model.fit(X.iloc[train_idx], y.iloc[train_idx])


In [482]:
pred = model.predict(X.iloc[[-1]])[0]

entry_condition = (
    pred != 0 and
    abs(df['NIFTY_Change'].iloc[-1]) > 5 and
    df['VIX_Close'].iloc[-1] > 14 and
    df['VIX_Change'].iloc[-1] > 0 and
    35 < df['RSI'].iloc[-1] < 65
)


In [483]:
if entry_condition:
    if pred == 1:
        signal = "BUY_CE"
    elif pred == -1:
        signal = "BUY_PE"
else:
    signal = "NO_TRADE"


In [484]:
vix = df['VIX_Close'].iloc[-1]

sl_points = max(15, vix * 0.8)
target_points = sl_points * 2


In [485]:
df = df.sort_values('Datetime').reset_index(drop=True)

df['signal'] = 0

buy_ce = (
    (df['EMA14'] > df['EMA20']) &
    (df['RSI'].between(40, 60)) &
    (df['NIFTY_Change'] > 5) &
    (df['VIX_Change'] > 0)
)

buy_pe = (
    (df['EMA14'] < df['EMA20']) &
    (df['RSI'].between(40, 60)) &
    (df['NIFTY_Change'] < -5) &
    (df['VIX_Change'] > 0)
)

df.loc[buy_ce, 'signal'] = 1
df.loc[buy_pe, 'signal'] = -1


In [486]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# ===============================
# 1. PREPARE DATA
# ===============================
df = df.sort_values('Datetime').reset_index(drop=True)
df['Datetime'] = pd.to_datetime(df['Datetime'])

# ===============================
# 2. CREATE SIGNAL COLUMN
# ===============================
df['signal'] = 0

buy_ce = (
    (df['EMA14'] > df['EMA20']) &
    (df['RSI'].between(40, 60)) &
    (df['NIFTY_Change'] > 5) &
    (df['VIX_Close'] > 14) &
    (df['VIX_Change'] > 0)
)

buy_pe = (
    (df['EMA14'] < df['EMA20']) &
    (df['RSI'].between(40, 60)) &
    (df['NIFTY_Change'] < -5) &
    (df['VIX_Close'] > 14) &
    (df['VIX_Change'] > 0)
)

df.loc[buy_ce, 'signal'] = 1
df.loc[buy_pe, 'signal'] = -1

# ===============================
# 3. TAKE LAST N CANDLES
# ===============================
df_plot = df.tail(80).copy()

# ===============================
# 4. SUBPLOTS
# ===============================
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.05,
    subplot_titles=("NIFTY Price + VWAP + Buy/Sell Signals", "RSI")
)

# ===============================
# 5. PRICE (CLOSE)
# ===============================
fig.add_trace(
    go.Scatter(
        x=df_plot['Datetime'],
        y=df_plot['close'],
        name='Close',
        line=dict(color='white', width=2)
    ),
    row=1, col=1
)

# ===============================
# 6. VWAP LINE (NEW)
# ===============================
fig.add_trace(
    go.Scatter(
        x=df_plot['Datetime'],
        y=df_plot['VWAP'],
        name='VWAP',
        line=dict(color='orange', width=2, dash='dot')
    ),
    row=1, col=1
)

# ===============================
# 7. BUY CE
# ===============================
buy_df = df_plot[df_plot['signal'] == 1]
fig.add_trace(
    go.Scatter(
        x=buy_df['Datetime'],
        y=buy_df['close'],
        mode='markers',
        name='BUY CE',
        marker=dict(color='lime', size=12, symbol='triangle-up'),
        hovertemplate="BUY CE<br>Time: %{x}<br>Price: %{y}<extra></extra>"
    ),
    row=1, col=1
)

# ===============================
# 8. BUY PE
# ===============================
sell_df = df_plot[df_plot['signal'] == -1]
fig.add_trace(
    go.Scatter(
        x=sell_df['Datetime'],
        y=sell_df['close'],
        mode='markers',
        name='BUY PE',
        marker=dict(color='red', size=12, symbol='triangle-down'),
        hovertemplate="BUY PE<br>Time: %{x}<br>Price: %{y}<extra></extra>"
    ),
    row=1, col=1
)

# ===============================
# 9. RSI PANEL
# ===============================
fig.add_trace(
    go.Scatter(
        x=df_plot['Datetime'],
        y=df_plot['RSI'],
        name='RSI',
        line=dict(color='yellow')
    ),
    row=2, col=1
)

fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
fig.update_yaxes(range=[0, 100], row=2, col=1)

# ===============================
# 10. LAYOUT
# ===============================
fig.update_layout(
    height=900,
    template='plotly_dark',
    title="NIFTY Option Buying Strategy • VWAP + RSI + VIX Filter",
    xaxis_rangeslider_visible=False,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()


In [487]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# ===============================
# 1. PREPARE DATA
# ===============================
df = df.sort_values('Datetime').reset_index(drop=True)
df['Datetime'] = pd.to_datetime(df['Datetime'])

# ===============================
# 2. CREATE future_price (SAFE)
# ===============================
if 'future_price' not in df.columns:
    df['future_price'] = (
        df['close']
        + df['NIFTY_Change'].rolling(3).mean().fillna(0)
    )

# ===============================
# 3. CREATE SIGNAL COLUMN
# ===============================
df['signal'] = 0

buy_ce = (
    (df['close'] > df['VWAP']) &                 # Price above VWAP
    (df['EMA14'] > df['EMA20']) &                # Uptrend
    (df['RSI'] > 55) &                           # Momentum confirmed
    (df['NIFTY_Change'] > 0) &                   # Index supportive
    (df['VIX_Close'] > 12)                       # Enough volatility
)


buy_pe = (
    (df['close'] < df['VWAP']) &                 # Below VWAP
    (df['EMA14'] < df['EMA20']) &                # Downtrend
    (df['RSI'] < 45) &                           # Bear momentum
    (df['NIFTY_Change'] < 0) &                   # Index weak
    (df['VIX_Close'] > 12)
)

df.loc[buy_ce, 'signal'] = 1
df.loc[buy_pe, 'signal'] = -1


# ===============================
# 4. LAST N CANDLES
# ===============================
df_plot = df.tail(80).copy()

# ===============================
# 5. PLOT
# ===============================
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    row_heights=[0.7, 0.3],
    vertical_spacing=0.05,
    subplot_titles=("NIFTY Price + VWAP + Prediction + Signals", "RSI")
)

# Close
fig.add_trace(go.Scatter(
    x=df_plot['Datetime'], y=df_plot['close'],
    name='Close', line=dict(color='white', width=2)
), row=1, col=1)

# VWAP
fig.add_trace(go.Scatter(
    x=df_plot['Datetime'], y=df_plot['VWAP'],
    name='VWAP', line=dict(color='orange', dash='dot')
), row=1, col=1)

# Predicted price
fig.add_trace(go.Scatter(
    x=df_plot['Datetime'], y=df_plot['future_price'],
    name='Predicted Price',
    line=dict(color='magenta', dash='dash')
), row=1, col=1)

# BUY CE
buy_df = df_plot[df_plot['signal'] == 1]
fig.add_trace(go.Scatter(
    x=buy_df['Datetime'], y=buy_df['close'],
    mode='markers', name='BUY CE',
    marker=dict(color='lime', size=12, symbol='triangle-up')
), row=1, col=1)

# BUY PE
sell_df = df_plot[df_plot['signal'] == -1]
fig.add_trace(go.Scatter(
    x=sell_df['Datetime'], y=sell_df['close'],
    mode='markers', name='BUY PE',
    marker=dict(color='red', size=12, symbol='triangle-down')
), row=1, col=1)

# RSI
fig.add_trace(go.Scatter(
    x=df_plot['Datetime'], y=df_plot['RSI'],
    name='RSI', line=dict(color='yellow')
), row=2, col=1)

fig.add_hline(y=70, row=2, col=1, line_dash="dash")
fig.add_hline(y=30, row=2, col=1, line_dash="dash")
fig.update_yaxes(range=[0, 100], row=2, col=1)

fig.update_layout(
    height=900,
    template='plotly_dark',
    title="NIFTY Option Buying Strategy (VWAP + Prediction + RSI + VIX)",
    xaxis_rangeslider_visible=False
)

fig.show()


In [488]:
print("EMA14 > EMA20:", (df['EMA14'] > df['EMA20']).sum())
print("RSI 40-60:", df['RSI'].between(40, 60).sum())
print("NIFTY_Change > 5:", (df['NIFTY_Change'] > 0.2).sum())
print("VIX_Change > 0:", (df['VIX_Change'] > 0.1).sum())


EMA14 > EMA20: 128
RSI 40-60: 145
NIFTY_Change > 5: 1
VIX_Change > 0: 0


In [490]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# --- Example: df should already have all your columns ---
# df = pd.read_csv('your_data.csv', parse_dates=['Datetime'])

# --- Step 1: Generate predicted price (simple placeholder model) ---
# For demonstration, let's use EMA14 shifted as a naive "future price"
df['future_price'] = df['EMA14'].shift(-1)

# --- Step 2: Generate Buy/Sell signals ---
# Buy: EMA14 crosses above EMA20 and RSI < 70
# Sell: EMA14 crosses below EMA20 and RSI > 30
df['signal'] = 0
df.loc[(df['EMA14'] > df['EMA20']) & (df['EMA14'].shift(1) <= df['EMA20'].shift(1)) & (df['RSI'] < 70), 'signal'] = 1
df.loc[(df['EMA14'] < df['EMA20']) & (df['EMA14'].shift(1) >= df['EMA20'].shift(1)) & (df['RSI'] > 30), 'signal'] = -1

# --- Step 3: Prepare figure ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=("Price + EMA + VWAP + Predicted + Signals", "RSI")
)

# --- PRICE + EMA + VWAP + Predicted ---
fig.add_trace(go.Scatter(x=df['Datetime'], y=df['close'], name='Close', line=dict(color='white')), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Datetime'], y=df['VWAP'], name='VWAP', line=dict(color='orange', dash='dot')), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Datetime'], y=df['EMA14'], name='EMA14', line=dict(color='cyan')), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Datetime'], y=df['EMA20'], name='EMA20', line=dict(color='magenta')), row=1, col=1)
fig.add_trace(go.Scatter(x=df['Datetime'], y=df['future_price'], name='Predicted Price', line=dict(color='yellow', dash='dash')), row=1, col=1)

# --- BUY SIGNALS ---
fig.add_trace(go.Scatter(
    x=df[df['signal'] == 1]['Datetime'],
    y=df[df['signal'] == 1]['close'],
    mode='markers',
    name='BUY',
    marker=dict(color='green', size=12, symbol='triangle-up')
), row=1, col=1)

# --- SELL SIGNALS ---
fig.add_trace(go.Scatter(
    x=df[df['signal'] == -1]['Datetime'],
    y=df[df['signal'] == -1]['close'],
    mode='markers',
    name='SELL',
    marker=dict(color='red', size=12, symbol='triangle-down')
), row=1, col=1)

# --- RSI ---
fig.add_trace(go.Scatter(x=df['Datetime'], y=df['RSI'], name='RSI', line=dict(color='yellow')), row=2, col=1)
fig.add_hline(y=70, line_dash="dash", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", row=2, col=1)

# --- Layout ---
fig.update_layout(
    height=900,
    template='plotly_dark',
    xaxis_rangeslider_visible=False,
    title="NIFTY Option Strategy Dashboard"
)

fig.show()
